# Core APIs

These are backend functions to help login, query, and store Plaid API data

In [2]:
#|default_exp core

In [3]:
#| export
import os, uuid, json, requests, psycopg2
import pandas as pd
from sqlalchemy import create_engine, Table, MetaData
from sqlalchemy.dialects.postgresql import insert 

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from typing import Optional, Dict, Any, List, Tuple

In [4]:
#|export
PLAID_COUNTRY_CODES = ['CA','US']
PLAID_PRODUCTS = ['transactions']
PLAID_CLIENT_ID= os.environ['PLAID_CLIENT_ID']
PLAID_SECRET= os.environ['PLAID_SECRET']
PLAID_ENV = os.environ['PLAID_ENV']
PLAID_BASE_URL = f'https://{PLAID_ENV}.plaid.com'

POSTGRES_DB="finances"
POSTGRES_HOST=os.environ['POSTGRES_HOST']
POSTGRES_USER= os.environ['POSTGRES_USER']
POSTGRES_PASSWORD= os.environ['POSTGRES_PASSWORD']
POSTGRES_ENCRYPTION_KEY=os.environ['POSTGRES_ENCRYPTION_KEY']
SQL_ENGINE =  f"postgresql+psycopg2://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}/{POSTGRES_DB}"


## Calculation Functions

In [22]:
#|export

def calculate_end_date(
        start_date: datetime, # (datetime): The starting date for the budget batch.
        budget_details: dict # (dict): Dictionary containing budget details, including cadence and refresh day.
) -> datetime: # datetime: The calculated end date.

    """
    Calculates the end date based on the budget details (refresh cadence and day of the week).
    """
    cadence = budget_details.get('refresh_cadence', '').lower()
    refresh_day_of_week = budget_details.get('refresh_day_of_week', '').lower()

    # Map the refresh_day_of_week to a number (Monday=0, Sunday=6)
    days_of_week = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
    target_day = days_of_week.index(refresh_day_of_week)
    
    # Calculate the number of days until the next target day
    days_until_target = (target_day - start_date.weekday() + 7) % 7
    if days_until_target == 0:  # If today is the target day, move to the next occurrence
        days_until_target = 7
    
    # Calculate the end date based on the cadence
    if cadence == 'weekly':
        end_date = start_date + timedelta(days=days_until_target)
    elif cadence == 'biweekly':
        end_date = start_date + timedelta(days=days_until_target + 7)  
    elif cadence == 'monthly':
            # Determine the next month's same-day occurrence
            next_month = start_date.month + 1
            next_year = start_date.year + (1 if next_month > 12 else 0)
            next_month = 1 if next_month > 12 else next_month
            
            # Set initial end date to the first of the next month
            end_date = datetime(next_year, next_month, 1)
            
            # Find the first occurrence of the target weekday in the new month
            while end_date.weekday() != target_day:
                end_date += timedelta(days=1)

    return end_date


In [23]:
#| hide

def test_calculate_end_date_weekly():
    """Test weekly cadence end date calculation."""
    start_date = datetime(2025, 2, 3)  # Monday
    budget_details = {"refresh_cadence": "weekly", "refresh_day_of_week": "fri"}
    expected_end_date = datetime(2025, 2, 7)  # Next Friday
    assert calculate_end_date(start_date, budget_details) == expected_end_date

def test_calculate_end_date_biweekly():
    """Test biweekly cadence end date calculation."""
    start_date = datetime(2025, 2, 3)  # Monday
    budget_details = {"refresh_cadence": "biweekly", "refresh_day_of_week": "fri"}
    expected_end_date = datetime(2025, 2, 14)  # Two Fridays later
    assert calculate_end_date(start_date, budget_details) == expected_end_date

def test_calculate_end_date_monthly():
    """Test monthly cadence end date calculation."""
    start_date = datetime(2025, 1, 15)  # Wednesday
    budget_details = {"refresh_cadence": "monthly", "refresh_day_of_week": "fri"}
    expected_end_date = datetime(2025, 2, 7)  # First Friday of the next month
    actual_end_date = calculate_end_date(start_date, budget_details)
    
    assert actual_end_date == expected_end_date

def test_calculate_end_date_same_day():
    """Test when the start date is already on the target refresh day."""
    start_date = datetime(2025, 2, 7)  # Friday
    budget_details = {"refresh_cadence": "weekly", "refresh_day_of_week": "fri"}
    expected_end_date = datetime(2025, 2, 14)  # Next Friday
    assert calculate_end_date(start_date, budget_details) == expected_end_date


In [24]:
#| hide

test_calculate_end_date_weekly()
test_calculate_end_date_biweekly()
test_calculate_end_date_monthly()
test_calculate_end_date_same_day()

Expected: 2025-02-07 00:00:00, Actual: 2025-02-07 00:00:00


## API Calls 

In [6]:
#|export

def plaid_post(
    endpoint: str, # The specific Plaid API endpoint (e.g., "accounts/get"), refer to [Plaid API Docs](https://plaid.com/docs/api/)
    payload: Dict[str, Any], # The JSON payload to be sent with the request
) -> Dict[str, Any]: # Returns JSON response from Plaid API
    """
    Makes a POST request to the Plaid API.
    """
    url = f"{PLAID_BASE_URL}/{endpoint}"  
    headers = {'Content-Type': 'application/json'} 
    response = requests.post(url, headers=headers, data=json.dumps(payload))  
    return response.json()  


def get_account(
    access_token: str  # The Plaid access token for the user account
) -> Dict[str, Any]: # Returns Dictionary object of accounts
    """
    Retrieves account details from the Plaid API.
    """
    payload = {
        "client_id": PLAID_CLIENT_ID,  
        "secret": PLAID_SECRET,        
        "access_token": access_token   
    }
    accounts_response = plaid_post("accounts/get", payload)
    return accounts_response  


def get_account_transactions(
    access_token: str,  # The Plaid access token for the user account
    start_date: str,    # The starting date for transactions (YYYY-MM-DD)
    end_date: str       # The ending date for transactions (YYYY-MM-DD)
) -> List[Dict[str, Any]]: # Returns List objects containing Dictionary objects related to account transactions
    """
    Retrieves all transactions for an account from the Plaid API.
    """
    try:
        transactions = []  # Initialize an empty list to hold transactions
        payload = {
            "client_id": PLAID_CLIENT_ID, 
            "secret": PLAID_SECRET,       
            "access_token": access_token,  
            "start_date": start_date,      
            "end_date": end_date,          
            "options": {
                "count": 100,  # Pagination
                "offset": 0   
            }
        }

        while True:
            transactions_response = plaid_post("transactions/get", payload)  
            transactions.extend(transactions_response.get("transactions", [])) 
            
            if len(transactions) >= transactions_response.get("total_transactions", 0):
                break
            
            payload["options"]["offset"] += 100

        return transactions  
    except Exception as e:
        print(f"An error occurred in get_account_transactions() \n{e}")
        raise


## Dataframe Transformation

In [7]:
#|export

def get_account_df(
    accounts_response: dict # Dictionary object containing accounts
) -> pd.DataFrame: # Returns  Dataframe of individual account
    """
    Converts account information from the Plaid API response into a pandas DataFrame.
    """
    return pd.json_normalize(accounts_response, sep='_')



def get_accounts_df(
    access_tokens: List[Tuple[str]] # List object containing access tokens
) -> pd.DataFrame: # Returns Dataframe of accounts
    """
    Retrieves and merges account information for multiple access tokens into a single pandas DataFrame.
    """
    accounts_df_list = []

    for single_access_token in access_tokens:
        account_data = get_account_df(get_account(single_access_token[0])['accounts'])
        accounts_df_list.append(account_data)
    return pd.concat(accounts_df_list, ignore_index=True)


def get_transactions_df(
    access_tokens: List[Tuple[str]],  # List object containing access tokens
    start_date: str = None,           # Optional start date (YYYY-MM-DD)
    end_date: str = None              # Optional end date (YYYY-MM-DD)
) -> pd.DataFrame:
    """
    Retrieves and converts transaction data for multiple access tokens into a pandas DataFrame.
    """
    transactions_list = []
    if not start_date:
        start_date = (datetime.today() - relativedelta(years=1)).strftime('%Y-%m-%d')
        end_date = datetime.today().strftime('%Y-%m-%d')
    print(f"Starting to get all transactions between {start_date} and {end_date}")
    for single_access_token in access_tokens:
        transactions_list.extend(get_account_transactions(single_access_token[0], start_date, end_date))

    return pd.json_normalize(transactions_list)


## Database Operations


In [8]:
#|export

def db_conn(
) -> psycopg2.extensions.connection:  # psycopg2 connection to database
    """
    Creates and returns a connection to the PostgreSQL database.
    """
    try:
        return psycopg2.connect(
            host=POSTGRES_HOST,
            database="finances",
            user=POSTGRES_USER,
            password=POSTGRES_PASSWORD
        )
    except psycopg2.OperationalError as e:
        print(f"Database connection skipped: {e}")
        return None


def db_sql(
    query: str  # The string representation of SQL query to execute
) -> pd.DataFrame: # Dataframe of executed SQL query
    """
    Executes a defined SQL query and returns the result as a pandas DataFrame.
    """
    engine = create_engine(SQL_ENGINE)
    try:
        return pd.read_sql_query(query, engine)
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()
    finally:
        engine.dispose()


def get_stored_public_access_tokens() -> List[Tuple[str]]: # Returns List Object with access tokens
    """
    Retrieves distinct Plaid access tokens from the local database.
    """
    try:
        db = db_conn()
        if not db:
            return []

        cur = db.cursor()
        # Decrypt the access token using pgp_sym_decrypt
        query = """
            SELECT DISTINCT 
                pgp_sym_decrypt(plaid_access_token::bytea, %s) AS decrypted_access_token
            FROM accounts;
        """
        cur.execute(query, (str(POSTGRES_ENCRYPTION_KEY),))
        tokens = cur.fetchall()
        cur.close()
        db.close()
        print(f"Found {len(tokens)} accounts and their access tokens.")
        return tokens
    except Exception as e:
        print(f"There was an error in get_stored_public_access_tokens():\n{e}")
        return []

def insert_account_df(
    access_token: str,  # The Plaid access token for the account
    accounts_response: dict,  # The response from the Plaid API containing account information
    email: str, # The user email
    phone: str, # The phone number associated with user
) -> None:
    """
    Inserts account information and the associated access token into the database.
    """
    try:
        accounts_df = get_account_df(accounts_response['accounts'])
        db = db_conn()
        if not db:
            return

        cur = db.cursor()
        for _, account in accounts_df.iterrows():
            cur.execute("""
                INSERT INTO accounts (
                    account_id, 
                    mask,
                    name,
                    official_name,
                    persistent_account_id,
                    subtype,
                    type,
                    user_email,
                    user_phone,
                    plaid_access_token
                ) 
                VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, pgp_sym_encrypt(%s::text, %s)
                )
                ON CONFLICT (account_id) 
                DO UPDATE SET
                    mask = EXCLUDED.mask,
                    name = EXCLUDED.name,
                    official_name = EXCLUDED.official_name,
                    persistent_account_id = EXCLUDED.persistent_account_id,
                    subtype = EXCLUDED.subtype,
                    type = EXCLUDED.type,
                    user_email = EXCLUDED.user_email,
                    user_phone = EXCLUDED.user_phone,
                    plaid_access_token = pgp_sym_encrypt(EXCLUDED.plaid_access_token::text, %s);
            """, (
                account['account_id'],
                account['mask'],
                account['name'],
                account['official_name'],
                account['persistent_account_id'],
                account['subtype'],
                account['type'],
                email,
                phone,
                access_token,
                POSTGRES_ENCRYPTION_KEY,
                POSTGRES_ENCRYPTION_KEY
            ))

        db.commit()
        cur.close()
        db.close()
        print(f"Successfully updated {accounts_df.shape[0]} accounts into the database.")
    except Exception as e:
        print(f"There was an error in insert_account_df():\n{e}")


def insert_transactions_df(
    transactions_df: pd.DataFrame  # A DataFrame containing transaction data
) -> None:
    """
    Inserts transaction data into the database.
    """
    try:
        db = db_conn()
        if not db:
            return

        cur = db.cursor()
        for _, transaction in transactions_df.iterrows():
            cur.execute("""
                INSERT INTO transactions (
                    transaction_id,
                    account_id,
                    amount,
                    authorized_date,
                    category_id,
                    date,
                    iso_currency_code,
                    logo_url,
                    merchant_entity_id,
                    merchant_name,
                    name,
                    payment_channel,
                    pending,
                    personal_finance_category_icon_url,
                    website,
                    personal_finance_category_detailed,
                    personal_finance_category
                ) 
                VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
                )
                ON CONFLICT (transaction_id) 
                DO UPDATE SET
                    account_id = EXCLUDED.account_id,
                    amount = EXCLUDED.amount,
                    authorized_date = EXCLUDED.authorized_date,
                    category_id = EXCLUDED.category_id,
                    date = EXCLUDED.date,
                    iso_currency_code = EXCLUDED.iso_currency_code,
                    logo_url = EXCLUDED.logo_url,
                    merchant_entity_id = EXCLUDED.merchant_entity_id,
                    merchant_name = EXCLUDED.merchant_name,
                    name = EXCLUDED.name,
                    payment_channel = EXCLUDED.payment_channel,
                    pending = EXCLUDED.pending,
                    personal_finance_category_icon_url = EXCLUDED.personal_finance_category_icon_url,
                    website = EXCLUDED.website,
                    personal_finance_category_detailed = EXCLUDED.personal_finance_category_detailed,
                    personal_finance_category = EXCLUDED.personal_finance_category;
            """, (
                transaction['transaction_id'],
                transaction['account_id'],
                transaction['amount'],
                transaction['authorized_date'] if pd.notnull(transaction['authorized_date']) else None,
                transaction['category_id'],
                transaction['date'],
                transaction['iso_currency_code'],
                transaction['logo_url'] if pd.notnull(transaction['logo_url']) else None,
                transaction['merchant_entity_id'] if pd.notnull(transaction['merchant_entity_id']) else None,
                transaction['merchant_name'] if pd.notnull(transaction['merchant_name']) else None,
                transaction['name'],
                transaction['payment_channel'],
                transaction['pending'],
                transaction['personal_finance_category_icon_url'] if pd.notnull(transaction['personal_finance_category_icon_url']) else None,
                transaction['website'] if pd.notnull(transaction['website']) else None,
                transaction['personal_finance_category.detailed'],
                transaction['personal_finance_category.primary']
            ))

        db.commit()
        cur.close()
        db.close()
        print(f"Successfully inserted {transactions_df.shape[0]} transactions into the database.")
    except Exception as e:
        print(f"There was an error in insert_transactions_df():\n{e}")


def upsert_account_balances_df(
    accounts_df: pd.DataFrame  # A DataFrame containing account balance data
) -> None:
    """
    Inserts or updates account balance history in the database.
    """
    try:
        db = db_conn()
        if not db:
            return
        cur = db.cursor()
        for _, account in accounts_df.iterrows():
            cur.execute("""
                INSERT INTO accounts_balance_history (
                    account_id, 
                    balances_available,
                    balances_current,
                    balances_iso_currency_code,
                    balances_limit,
                    balances_unofficial_currency_code,
                    balances_datetime
                ) 
                VALUES (
                    %s, %s, %s, %s, %s, %s, %s
                );
            """, (
                account['account_id'],
                account['balances_available'],
                account['balances_current'],
                account['balances_iso_currency_code'],
                account['balances_limit'],
                account['balances_unofficial_currency_code'],
                datetime.now(),
            ))

        db.commit()
        cur.close()
        db.close()
        print(f"Successfully updated {accounts_df.shape[0]} account balances into the database as of {datetime.now()}")

    except Exception as e:
        print(f"There was an error in upsert_account_balances_df():\n{e}")

def get_last_successful_refresh( 
    )-> str: # Return last successful refresh of transaction data
    """
    Query database to check when the last successful refresh was
    """
    try:
        query = """ 
        SELECT id, refresh_time, refresh_status, refresh_description
            FROM public.fin_refresh
        WHERE refresh_status = TRUE
        ORDER BY refresh_time DESC
        LIMIT 1;
        """

        return (db_sql(query)).loc[0, 'refresh_time']
    except Exception as e:
        print(f"There was an error in get_last_successful_refresh():\n{e}")
        return []
    
def update_last_refresh(success:bool=True, # Assumes that refresh was successful
                        msg:str="Refresh was successful", # Refresh message
                        ref_type:str=None, # Type of refresh one of ['Transaction', 'Balance_Hist]
    )-> None:
    """
    Update database to log refresh time and status. If an error is thrown, the `msg` committed to database will print stack trace
    """
    try:
        # Define the SQL query
        query = """
        INSERT INTO public.fin_refresh (refresh_time, refresh_status, refresh_description, refresh_type)
        VALUES (%s, %s, %s, %s);
        """
        refresh_time = datetime.now()

        connection = db_conn()
        if connection is None:
            print("Failed to connect to the database. Cannot execute the query.")
            return

        with connection.cursor() as cursor:
            cursor.execute(query, (refresh_time, success, msg, ref_type))
            connection.commit()
            print("Refresh record successfully inserted into the database.")
    except Exception as e:
        print(f"There was an error in update_last_refresh():\n{e}")


def insert_new_budget(
        name: str, # String Budget Name
        description: str, # String Budget Description
        limit: float, # Maximum value for Budget
        cadence: str = "weekly", # String, one of ['weekly', 'biweekly', 'monthly', 'yearly']
        date_of_week: str ="sun", # String, one of [ 'mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun'] 
        rules: str=None # String, representing latter half of SQL query to define the types of transactions to be automatically associated
)-> None:
    """
    Insert a new budget record into the database.
    """
        
    # Validate the inputs
    if cadence not in ['weekly', 'biweekly', 'monthly']:
        raise ValueError("Invalid refresh cadence. Must be one of ['weekly', 'biweekly', 'monthly'].")
    if date_of_week not in ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']:
        raise ValueError("Invalid refresh day of week. Must be one of ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun'].")
    
    try:
        # Define the SQL query
        query = """
        INSERT INTO budget (name, description, balance_limit, refresh_cadence, refresh_day_of_week, rules, is_deleted)
        VALUES (%s, %s, %s, %s, %s, %s, False);
        """

        # Establish the database connection
        connection = db_conn()
        if connection is None:
            print("Failed to connect to the database. insert_new_budget()")
            return
        # Execute the query
        with connection.cursor() as cursor:
            cursor.execute(query, (name, description, limit, cadence, date_of_week, rules))
            connection.commit()
            print("Budget successfully inserted into the database.")
    except Exception as e:
        print(f"Error in insert_new_budget(): {e}")
    finally:
        # Ensure the connection is properly closed
        if connection:
            connection.close()

def get_all_active_budgets()-> pd.DataFrame:
    """
    Returns dataframe containig budgets that are active
    """
    return db_sql("SELECT * FROM budget WHERE is_deleted = false")

def get_budget_details(
        id:int # budget_id 
)-> Dict: # returns a dictionary object with budgetting details
    """
    Returns dictionary with budget detail information with provided `budget_id`
    """
    try:
        # Validate the ID is numeric
        if not isinstance(id, int):
            raise ValueError("The provided ID must be an integer.")
        
        query = f"SELECT * FROM budget WHERE id = {id};"
        result_df = db_sql(query)
        
        if result_df.empty:
            print(f"No budget found with ID: {id}")
            return {}
        
        budget_details = result_df.iloc[0].to_dict()
        return budget_details
    except Exception as e:
        print(f"Error in get_budget_details(): {e}")
        return {}   
    
def get_latest_budget_batch(
        id:int # budget_id 
)-> Dict: # returns a dictionary object with latest budget batch detail
    """
    Returns dictionary with latest budget batch information with provided `budget_id`
    """
    try:
        # Validate the ID is numeric
        if not isinstance(id, int):
            raise ValueError("The provided ID must be an integer.")
        
        query = f"SELECT * FROM public.budget_batch WHERE budget_id={id} ORDER BY end_date DESC LIMIT 1;"
        result_df = db_sql(query)
        if result_df.empty:
            print(f"No budget batch found with ID: {id}")
            return {}
        
        latest_batch = result_df.iloc[0].to_dict()
        return latest_batch
    except Exception as e:
        print(f"Error in get_latest_budget_batch(): {e}")
        return {}   


def insert_new_budget_batch(
        budget_id: int,  # Integer ID of the budget 
) -> None:
    """
    Insert a new budget batch record into the database.
    """
    # if a current budget batch exists - return
    latest_batch = get_latest_budget_batch(budget_id)
    budget_updated = get_budget_details(budget_id)['update_time'] 
    try:
        if latest_batch and (latest_batch['end_date'] > datetime.today() and budget_updated < latest_batch['start_date']):
            print(f"There is already a batch active for budget_id ({budget_id}), which ends on {latest_batch['end_date']}.")
            return
    except:
        pass

    try:
        # check what type of refresh the budget is
        budget_details =  get_budget_details(budget_id)
        # Start date will depend whether a batch has existed or not.
        start_date = datetime.today()
        if latest_batch != {}:
            print(f"The last batch ended on {latest_batch['end_date']}.")
            start_date=latest_batch['end_date']
        try:    
            if budget_updated > latest_batch['start_date']:
                print(f"The budget was updated on {get_budget_details(budget_id)['update_time']} Starting new batch.")
                start_date = datetime.today()
        except:
            pass
        end_date = calculate_end_date(start_date, budget_details)

        query = """
        INSERT INTO budget_batch (budget_id, start_date, end_date, current_balance, under_limit)
        VALUES (%s, %s, %s, %s, %s);
        """

        connection = db_conn()
        if connection is None:
            print("Failed to connect to the database. insert_new_budget_batch()")
            return
        
        with connection.cursor() as cursor:
            cursor.execute(query, (budget_id, start_date, end_date, 0, True))
            connection.commit()
            print(f"Budget batch ({budget_id}) successfully inserted into the database.")
    except Exception as e:
        print(f"Error in insert_new_budget_batch(): {e}")
    finally:
        if connection:
            connection.close()

def get_latest_batch_id(
        budget_id:int # integer of `budget_id`
)->int: # integer of `batch_id`
    """
    Lookup the latest `batch_id` associated with a `budget_id`
    """
    query = f"SELECT id from v_lastest_budget_batches WHERE budget_id={budget_id};"
    result_df = db_sql(query)
    if result_df.empty:
        print(f"No budget batch found with ID: {id}")
        return {}
    latest_batch = result_df.iloc[0].to_dict()
    return latest_batch

def insert_budgetted_transaction(
        batch_id: int,  # Integer ID of the batch
        transaction_id: str, # ID associated with transaction
) -> None:
    """
    Insert a new budgetted transaction into the database.
    """
    try:
        query = """
        INSERT INTO budgeted_transaction (batch_id, transaction_id)
        VALUES (%s, %s);
        """
        update_query = """
        UPDATE transactions
        SET budget_run = TRUE
        WHERE transaction_id = %s;
        """

        connection = db_conn()
        if connection is None:
            print("Failed to connect to the database. insert_budgetted_transaction()")
            return
        
        with connection.cursor() as cursor:
            cursor.execute(query, (batch_id, transaction_id))
            cursor.execute(update_query, (transaction_id,))

            connection.commit()
    except Exception as e:
        print(f"Error in insert_budgetted_transaction(): {e}")
    finally:
        if connection:
            connection.close()

def get_budget_rules( 
)-> Dict: # Dict item containing the budget rules associated with the latest batches
    """
    Query the database and return python dictionary of the rules associated with the latest batches
    """
    try:
        rules = {}
        # Fetch the latest budget batch IDs
        active_budgets = db_sql("SELECT id, budget_id FROM v_lastest_budget_batches;")
        
        if active_budgets.empty:
            print("No active budgets found.")
            return rules
        
        for _, budget in active_budgets.iterrows():
            budget_id = budget['budget_id']
            query = f"SELECT rules FROM budget WHERE id = {budget_id};"
            result = db_sql(query)
            
            if not result.empty and len(result) == 1:
                rules[budget_id] = result.iloc[0]['rules']
            else:
                print(f"No rules found for budget ID {budget_id}, or multiple results returned.")
        
        return rules
    except Exception as e:
        print(f"Error in get_budget_rules(): {e}")
        return {}

def run_sp_update_batch_balances()->None:
    """ 
    Runs stored procedure sp_update_batch_balances()

    """

    try:
        db = db_conn()
        cursor = db.cursor()

        cursor.execute("CALL sp_update_batch_balances()")
        db.commit()  

    except Exception as e:
        print(f"Error in run_sp_update_batch_balances(): {e}")

    finally:
        if db:
            db.close()


def upsert_dataframe_to_db(
        df: pd.DataFrame, # Data frame to be applied to database
        table_name: str,  # table name
        unique_key: list # primary key
) -> None:
    """
    Inserts or updates rows in a table based on the given DataFrame.

    This function assumes the DataFrame schema is exactly the same as the table.
    """

    engine = create_engine(SQL_ENGINE)
    metadata = MetaData()
    metadata.reflect(bind=engine)
    table = metadata.tables[table_name]

    rows = df.to_dict(orient='records')

    if not rows:
        print("DataFrame is empty. No operation performed.")
        return

    with engine.connect() as conn:
        trans = conn.begin() 
        try:
            for row in rows:
                # Build the INSERT statement
                stmt = insert(table).values(row)
                update_dict = {
                    col.name: stmt.excluded[col.name]
                    for col in table.columns
                    if col.name not in unique_key
                }

                # Add ON CONFLICT clause for upsert
                stmt = stmt.on_conflict_do_update(
                    index_elements=unique_key,
                    set_=update_dict
                )
                conn.execute(stmt)

            trans.commit()  
        except Exception as e:
            print(f"Error in writing to DB upsert_dataframe_to_db(): {e}")
            trans.rollback()

## Authentication

In [9]:
#|export

def generate_link_token(
    email: str,  # The user's email address
    phone: str   # The user's phone number
) -> Optional[str]:  # Returns the generated link token if successful, otherwise None
    """
    Generates a link token to authenticate with the Plaid API.
    """
    try:
        payload = {
            "client_id": PLAID_CLIENT_ID,
            "secret": PLAID_SECRET,
            "client_name": "Jupyter Notebook",
            "country_codes": PLAID_COUNTRY_CODES,
            "language": "en",
            "user": {
                "client_user_id": str(uuid.uuid4()),
                "phone_number": phone,
                "email_address": email,
            },
            "hosted_link": {},
            "products": PLAID_PRODUCTS,
        }
        link_token_response = plaid_post("link/token/create", payload)

        print(
            f"Navigate to this page and authenticate with your bank: "
            f"{link_token_response.get('hosted_link_url', 'No URL provided')}"
        )
        print(
            f"This link expires: {link_token_response.get('expiration', 'No expiration provided')}"
        )

        return link_token_response.get("link_token")

    except Exception as e:
        print(f"There was an error in generate_link_token:\n{e}")
        return None


def get_and_save_public_token(
    link_token: str, # The link token generated during the Plaid Link flow
    email: str,  # The user's email address
    phone: str   # The user's phone number
) -> None:
    """
    Retrieves a public token using the link token, exchanges it for an access token, 
    and saves the account information to the database.
    """
    exchange_response = ''
    access_token = ''

    try:
        payload = {
            "client_id": PLAID_CLIENT_ID,
            "secret": PLAID_SECRET,
            "link_token": link_token,
        }
        link_token_details = plaid_post("link/token/get", payload)

        public_token = (
            link_token_details['link_sessions'][0]['results']['item_add_results'][0]['public_token']
        )

        payload = {
            "client_id": PLAID_CLIENT_ID,
            "secret": PLAID_SECRET,
            "public_token": public_token,
        }
        exchange_response = plaid_post("item/public_token/exchange", payload)
        access_token = exchange_response.get("access_token")

        if access_token:
            print("Access token has been generated successfully")
        else:
            print("Failed to generate access token.")
            return

    except KeyError as ke:
        print(f"Key error occurred in get_and_save_public_token: Missing key {ke}")
        return
    except Exception as e:
        print(f"There was an error in get_and_save_public_token:\n{e}\nResponse: {exchange_response}")
        return

    try:
        accounts_response = get_account(access_token)
        insert_account_df(access_token, accounts_response, email, phone)
    except Exception as e:
        print(f"There was an error while saving account information in get_and_save_public_token:\n{e}")


## Tasks

In [10]:
get_budget_rules()

{np.int64(1): "personal_finance_category = 'TRANSPORTATION' OR personal_finance_category = 'TRAVEL'",
 np.int64(2): "personal_finance_category_detailed = 'FOOD_AND_DRINK_COFFEE'",
 np.int64(3): "personal_finance_category = 'ENTERTAINMENT' OR personal_finance_category = 'LOAN_PAYMENTS'",
 np.int64(4): "personal_finance_category_detailed = 'FOOD_AND_DRINK_FAST_FOOD' AND NOT merchant_name = 'KFC'",
 np.int64(5): "personal_finance_category_detailed = 'FOOD_AND_DRINK_FAST_FOOD' AND merchant_name = 'KFC'"}

In [11]:
#|export

def run_budgetting_rules()->None:
    """
    Automatically assign transactions to budgets based on pre-determined rules.
    These rules are not yet 'validated'
    """
    run_sp_update_batch_balances()

    rules = get_budget_rules()
    for rule in rules:
        batch_id = get_latest_batch_id(rule)['id']
        transactions = db_sql(f"SELECT * from transactions WHERE ({rules[rule]}) AND budget_run = False;")
        for _, record in transactions.iterrows():
            insert_budgetted_transaction(batch_id, record['transaction_id'])
    
        print(f"Applied {transactions.shape[0]} transactions to the rule:\n     {rules[rule]}")

def run_budgetting_tasks() -> None:
    budget_df = get_all_active_budgets()

    for _, budget in budget_df.iterrows():
        insert_new_budget_batch(budget["id"])
        
    run_budgetting_rules()
    run_sp_update_batch_balances()
    print("Validated all budgetting batches for today.")

        
def get_and_save_all_account_transactions(first_time=False) -> None:
    """
    Retrieves all account transactions for stored public access tokens
    and inserts the transactions into the database.

    Steps:

    * Fetch public access tokens from the database.
    * Retrieve transactions data for each account associated with the tokens. (2 cases, First time load or Incremental) 
    * Insert the retrieved transactions into the database.

    Returns:
        None
    """
    try:
        # Step 1: Fetch public access tokens
        access_tokens = get_stored_public_access_tokens()
        # Step 2: Retrieve all transactions as a DataFrame
        # Step 2A: First Time Load
        if first_time:
            transactions_df = get_transactions_df(access_tokens)
            insert_transactions_df(transactions_df)
            update_last_refresh(ref_type="Transaction")

        # Step 2B: Incremental load
        else:
            start_date = get_last_successful_refresh().strftime('%Y-%m-%d')
            start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
            today_date = datetime.today()
            date_diff = (today_date - start_date_obj).days
            if date_diff >= 1:
                start_date = get_last_successful_refresh().strftime('%Y-%m-%d')
                transactions_df = get_transactions_df(access_tokens, start_date, datetime.today().strftime('%Y-%m-%d'))
                insert_transactions_df(transactions_df)
                print("Successfully retrieved and saved all account transactions.")
                update_last_refresh(ref_type="Transaction")
            else:
                print("Data was already refreshed today.")

    except Exception as e:
        update_last_refresh(False, e, ref_type="Transaction")
        print(f"An error occurred in get_and_save_all_account_transactions:\n{e}")


def get_and_save_balance_history() -> None:
    """
    Retrieves account balance history for stored public access tokens
    and updates the balance history in the database.

    Steps:
    
    * Fetch public access tokens from the database.
    * Retrieve account details for each account associated with the tokens. 
    * Update the account balance history in the database. 
    
    Returns:
        None
    """
    try:
        # Step 1: Fetch public access tokens
        access_tokens = get_stored_public_access_tokens()

        # Step 2: Retrieve all account details as a DataFrame
        accounts_df = get_accounts_df(access_tokens)

        # Step 3: Update account balance history in the database
        upsert_account_balances_df(accounts_df)
        update_last_refresh(ref_type="Balance_Hist")
    except Exception as e:
        print(f"An error occurred in get_and_save_balance_history()\n{e}")


In [12]:
#|export

def about():
    """
    Print environmental details for this instance of `jupyter-finance`
    """
    print("="*60)
    print("="*60)
    print(f"Jupyter Finances")
    print(f"Current Date and Time: {datetime.now()}")
    
    print("="*60)
    print(f"Plaid API ({PLAID_ENV})")
    print("="*60)
    print(f"PLAID_CLIENT_ID: {PLAID_CLIENT_ID}")
    print(f"PLAID_PRODUCTS: {PLAID_PRODUCTS}")
    print(f"PLAID_COUNTRY_CODES: {PLAID_COUNTRY_CODES}")
    print("="*60)
